In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
pip install pychord

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import os
import numpy as np
import sys
import glob
import random
import torch
from torch import nn
import torch.nn.functional as F
import math
import time
import datetime
import json
from pychord import Chord, find_chords_from_notes

In [41]:
# # ♭ to b
# path = '/content/drive/MyDrive/Colab Notebooks/0sh/chord_gen/song_json/song_json'
# files = glob.glob(path + '/*.json')
# for file in files:
#     with open(file, 'r', encoding='utf-8') as f:
#         print(file)
#         content = json.load(f)
#         print(content)
#         content['Chords'] = content['Chords'].replace('♭', 'b')
#     file = file.replace('song_json/song_json', 'song_json/song_json_b')
#     with open(file.replace('.json', '_b.json'), 'w', encoding='utf-8') as f:
#         json.dump(content, f)
#         print(content)

In [42]:
c = Chord('Cm')
print(c.components())
print(c.quality)
print(c.root)

['C', 'Eb', 'G']
m
C


In [43]:
# def random_sample_file_test():
#     files = glob.glob('/content/drive/MyDrive/Colab Notebooks/0sh/chord_gen/song_json/song_json' + '/*.json')
#     print(len(files))
#     print(files[0])
#     random_sample_file = random.sample(files, 10)
#     chords = [0]*10
#     for i, file in enumerate(random_sample_file):
#         with open(file) as f:
#             chords[i] = json.load(f)
#     print(chords[0]['Chords'])

# random_sample_file()

# type(Chord('m'))

m = Chord('C')
m.transpose(13)
print(m)
"Em".split()[0]


Db


'Em'

In [44]:
class GRU_LM(nn.Module):
    def __init__(self, input_dim, hidden_dim, target_dim, num_layers):
        super (GRU_LM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.target_dim = target_dim
        self.num_layers = num_layers
        self.ln1 = nn.Linear(self.input_dim, self.input_dim*4)
        self.gru = nn.GRU(self.input_dim*8, self.hidden_dim, num_layers=self.num_layers, batch_first=True, bidirectional=False)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.1)
        self.hidden2target  = nn.Linear(self.hidden_dim, self.target_dim)
        self.sigmoid = nn.Sigmoid()

        self.conv11 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), padding=(1,1))
        self.conv12 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=(1,1))
        self.bn1 = nn.BatchNorm2d(16, affine=True)
        self.pool1 = nn.MaxPool2d(kernel_size=(4,1))
        self.conv21 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=(1,1))
        self.conv22 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=(1,1))
        self.bn2 = nn.BatchNorm2d(32, affine=True)
        self.pool2 = nn.MaxPool2d(kernel_size=(4,1))

    def extract_feature(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = self.bn1(x)
        x = self.pool1(x)
        x = F.relu(self.conv21(x))
        x = F.relu(self.conv22(x))
        x = self.bn2(x)
        x = self.pool2(x)
        return x

    def forward(self, x):
        '''
            batch * chroma vec * length
            B * 12 * T -> B * 12 * T
        '''
        x = x.permute(0,2,1)
        x = self.ln1(x)
        x = x.permute(0,2,1)
        x = torch.unsqueeze(x, 1)
        x = self.extract_feature(x)
        x = self.dropout1(x)
        x = x.view(-1, x.size(1)*x.size(2), x.size(3)).permute(0,2,1)
        x = F.elu(x)
        self.gru.flatten_parameters()
        x, h_n = self.gru(x)
        x = self.dropout2(x)
        x = self.hidden2target(x).permute(0,2,1)
        x = self.sigmoid(x)
        return x


In [45]:
def LM_train(x, model, device):
    x_mov = torch.cat([torch.zeros(x.size(0),x.size(1),1).to(device), x[:,:,:-1]], axis=2).float().to(device)
    y = model(x_mov)
    # print(x_mov[0,:,:15])
    # print(x[0,:,:15])
    x = x.permute(0,2,1)
    y = y.permute(0,2,1)
    criterion = nn.functional.binary_cross_entropy
    return criterion(y,x)

In [46]:
def key2Cmaj(key, chord):
    '''
        Chord, list(Chord)
        do not distinguish major and minor
        -> wanna do...
    '''
    for tran in range(12):
        if Chord(key.root) == Chord('C'): break
        key.transpose(1)
    if str(key.quality) == 'm': tran -= 3
    tran = tran % 12
    for c in chord: c.transpose(tran)
    chord_modified = chord.copy()
    return chord_modified

In [47]:
def note2cv(note_chord):
    for i in range(12):
        note_number = Chord('C')
        note_number.transpose(i)
        if Chord(note_chord) == note_number: break
    cv = np.zeros((12,1))
    cv[i, 0] = 1
    return cv

In [48]:
def c2cv(chord):
    '''
        chord -> chroma vector <ndarray>
    '''
    note_list = chord.components()
    cv = np.zeros((12,1))
    for note in note_list:
        cv += note2cv(note)
    return cv

In [49]:
def random_sample_file(INPUT_DIR, batchsize):
    files = glob.glob(INPUT_DIR + '/*_b.json')
    random_sample_file = random.sample(files, batchsize)
    songs = []
    for i, file in enumerate(random_sample_file):
        try:
            with open(file) as f:
                songs.append(json.load(f))
        except:
            print('Error in random_sample_file')
    return songs

In [50]:
def load_chords(batchsize):
    '''
        return Chord, list(Chord)
    '''
    path = '/content/drive/MyDrive/Colab Notebooks/0sh/chord_gen/song_json/song_json_b'
    songs = random_sample_file(path, batchsize)
    keys = []
    chords = []
    for i in range(len(songs)):
        try:    # .split()[0]の理由は'Play':'Em　　＜標準コード譜ページに戻る＞'とかあるから
            if songs[i]['Play'].split()[0] == 'None' or songs[i]['Play'].split()[0] == 'm': continue
            if len(songs[i]['Chords'].split()) < 10: continue # over 10 chords sequence 
            keys.append(Chord(songs[i]['Play'].split()[0]))
            chords.append([Chord(c) for c in songs[i]['Chords'].split()])
        except:
            print('This has Play: ' + songs[i]['Play'])
    return keys, chords

In [51]:
cd drive/MyDrive/Colab\ Notebooks/0sh/chord_gen/params

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/0sh/chord_gen/params'
/content/drive/MyDrive/Colab Notebooks/0sh/chord_gen/params


In [55]:
epoch_num = 50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batchsize = 5
z_dim = 512

lr = 1e-3
weight_decay = 1e-4
betas = (0.9, 0.98)
eps = 1e-09
step_size = 1
d_adam = {"lr": lr, "betas": betas, "eps": eps, "weight_decay": weight_decay}

d_lang = {"input_dim":12, "hidden_dim":128, "target_dim":12, "num_layers":5}   # クロマベクトルdim12
model = GRU_LM(**d_lang)
model.to(device)

load_weights = torch.load('LM1.7.pth')
model.load_state_dict(load_weights)

optimizer = torch.optim.AdamW(model.parameters(), **d_adam)

# time
timenow = 0

# eatlystopping
ESpatience = 10
EScounter = 0
ESpreloss = 1000  # any big number

for epoch in range(1, epoch_num+1):

    # rest of the time estimate
    restime = (epoch_num-epoch)*(time.time()-timenow)/3600
    resthour = int(restime)
    restmin = int((restime-resthour)*60)
    print("About " + str(resthour) + "h " + str(restmin) + "m remaining")
    timenow = time.time()
    
    epoloss = {'chord':[], 'beat':[], 'down':[]}

    # batch number
    bat = 1

    iteration = int(10000/batchsize)
    iteration = 10

    for iteNum in range(iteration):
        # show rest
        pro_bar = ('=' * int((iteNum+1)/10)) + (' ' * int((iteration - (iteNum+1))/10))
        print('\r[{0}] {1}/{2}'.format(pro_bar, (iteNum+1), iteration), end='')

        # data load randomly
        keys, chords = load_chords(batchsize)

        # len(key) <= batchsize
        rbatchsize = min(len(keys),len(chords))
        # print('  ' + str(rbatchsize) + '   ')

        # no consective same chord
        for b in range(rbatchsize):
            chords[b] = noCon(chords[b])

        # set key to Cmaj
        for b in range(rbatchsize):
            chords[b] = key2Cmaj(keys[b], chords[b])
            # print(chords[b])

        # chord to chroma vector
        chroma = [0]*rbatchsize
        for b in range(rbatchsize):
            chroma[b] = [c2cv(chord) for chord in chords[b]]
        
        # to ndarray
        len_list = [len(b) for b in chords]
        min_len = min(len_list)
        # print('  ' + str(min_len) + '   ')
        for b in range(rbatchsize):
            chroma[b] = np.concatenate(chroma[b][:min_len], axis=1)
        chroma = np.stack(chroma)

        # to tensor
        chroma = torch.Tensor(chroma).to(device)
        # print(chroma[0,:,:10])

        # forward
        loss = LM_train(chroma, model, device)
        # print(chroma[0][:,:10])

        # update Ws
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # save
        epoloss['chord'].append(loss.cpu().data.numpy().tolist())


        # VAE的なやつも作る


    lossmean_drum = float(np.mean(epoloss['chord']))
    print(" ")
    print(str(epoch) + " " + str(lossmean_drum))

    # earlystopping
    if lossmean_drum * 1.01 > ESpreloss:
        EScounter += 1
    else:
        EScounter = 0
    if EScounter == ESpatience:
        break
    ESpreloss = lossmean_drum


# output nn
# /content/drive/Mydrive/Colab Notebooks/0sh/chord_gen/params/
nn_save_path = 'LM1.8.pth'
torch.save(model.state_dict(), nn_save_path)


About 22609414h 25m remaining
[=] 10/10 
1 0.09049871414899827
About 1h 8m remaining
[] 1/10This has Play: D♭
[] 7/10This has Play: G♭　（曲中で転調しているので、オリジナルキーで掲載しております）
[] 9/10This has Play: －
[=] 10/10 
2 0.07414549812674523
About 1h 12m remaining
[=] 10/10 
3 0.07563721109181643
About 0h 37m remaining
[=] 10/10 
4 0.05057619903236628
About 0h 45m remaining
[=] 10/10 
5 0.04516248879954219
About 0h 31m remaining
[=] 10/10 
6 0.05409635817632079
About 1h 1m remaining
[=] 10/10 
7 0.04380384311079979
About 1h 27m remaining
[=] 10/10 
8 0.048213633522391316
About 0h 44m remaining
[=] 10/10 
9 0.04557267539203167
About 0h 44m remaining
[=] 10/10 
10 0.03655221648514271
About 0h 26m remaining
[=] 10/10 
11 0.04920308422297239
About 1h 4m remaining
[=] 10/10 
12 0.03832772700116038
About 0h 46m remaining
[] 3/10This has Play: E♭
[=] 10/10 
13 0.047433755919337274
About 0h 20m remaining
[=] 10/10 
14 0.03985461313277483
About 0h 41m remaining
[=] 10/10 
15 0.04391246885061264
About 0h 26m remai

パラメータとモデルの対応
*   LM1.1.pth {"input_dim":12, "hidden_dim":64, "target_dim":12, "num_layers":4}
*   LM1.2.pth {"input_dim":12, "hidden_dim":128, "target_dim":12, "num_layers":5}
*   LM1.5.pth 諸々変更（連続コードを削除）
*   LM.1.6.pth CNN追加 loss : 0.098いい感じ
*   LM.1.7.pth 1.6を更に学習 loss : 0.048
*   LM.1.8.pth loss : 0.026